## Deepfake Images

### Importing Libraries

In [ ]:
# Data Processing
import numpy as np
import pandas as pd
import scipy.misc
from sklearn.datasets import load_files 
import matplotlib.pyplot as plt
%matplotlib inline

# Deep Learning Libraries
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Flatten, Dropout, Activation, Lambda, Permute, Reshape
from keras.layers import Convolution2D, ZeroPadding2D, MaxPooling2D
from keras_vggface.vggface import VGGFace
from keras_vggface import utils
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils


### Data Loading

In [ ]:
training_data_path = ""
test_data_path = ""
batch_size = 64

print(“Loading Train…”)
training_data = ImageDataGenerator(rescale = 1./255.) 
                .flow_from_directory(
                        training_data_path,
                        target_size=(224, 224),
                        batch_size=batch_size,
                        class_mode='binary'
                )


print(“Loading Test…”)
test_data = ImageDataGenerator(rescale = 1./255.)
            .flow_from_directory(
                        test_data_path,
                        target_size=(224, 224),
                        batch_size=batch_size,
                        class_mode='binary'
            )

### CNN Model

In [ ]:
input_shape = (224,224,3)
epsilon=0.001
dropout = 0.1
model = Sequential()

# Convolution and Pooling
model.add(BatchNormalization(input_shape=input_shape))
model.add(Conv2D(filters=16, kernel_size=3, activation='relu', padding=’same’))
model.add(MaxPooling2D(pool_size=2))
model.add(BatchNormalization(epsilon=epsilon))


model.add(Conv2D(filters=32, kernel_size=3, activation='relu', padding=’same’))
model.add(MaxPooling2D(pool_size=2))
model.add(BatchNormalization(epsilon=epsilon))
model.add(Dropout(dropout))

model.add(Conv2D(filters=64, kernel_size=3, activation='relu', padding=’same’))
model.add(MaxPooling2D(pool_size=2))
model.add(BatchNormalization(epsilon=epsilon))
model.add(Dropout(dropout))

# Aggregation
model.add(GlobalAveragePooling2D())

# Fully Connected Layer
model.add(Dense(1, activation='sigmoid'))


In [ ]:
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy',optimizer=Adam(0.0001), metrics=['accuracy'])

training_steps = 40000//batch_size
num_epochs = 10

history = model.fit_generator(
    training_data,
    epochs=num_epochs,
    steps_per_epoch = training_steps
)


In [ ]:
y_pred = model.predict(test_data)
y_actual = test_data.classes


## Deepfake Videos

### Importing Libraries

In [ ]:
# Libraries for Machine Learning
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np

# Helper Libraries
import imageio
import cv2
import os


### Data Loading

In [ ]:
train_meta_file = '../deepfake-detection-challenge/train_sample_videos/metadata.json'
train_sample_metadata = pd.read_json(train_meta_file).T
train_sample_metadata.head()

In [ ]:
def crop_image(frame):
    y = frame.shape[0] 
    x = frame.shape[1]
    min_dimension = min(x, y)
    start_x = (x/2) - (min_dimension/2)
    start_y = (y/2) - (min_dimension/2)
    cropped = frame[start_y : start_y + min_dim, 
                    start_x : start_x + min_dim]
    return cropped


In [ ]:
def parse_video_into_frames(video_file_path):
    new_shape = (224, 224)
    capture = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = capture.read()
            if not ret: # Have reached the end of frames
                break
            frame = crop_image(frame)
            frame = cv2.resize(frame, new_shape)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)
    finally:
        capture.release()
    return np.array(frames)


### Inception Model for Feature Extraction

In [ ]:
inception_model = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(224,224, 3),
    )

preprocess_input = keras.applications.inception_v3.preprocess_input
inputs = keras.Input((224,224, 3))
preprocessed = preprocess_input(inputs)
outputs = inception_model(preprocessed)
feature_extractor = keras.Model(inputs, outputs, name="feature_extractor")


### Data Preprocessing

In [ ]:
def prepare_data(df, data_dir):
    MAX_SEQ_LENGTH = 20
    NUM_FEATURES = 2048

    num_samples = len(df)
    video_paths = list(df.index)
    labels = df["label"].values
    labels = np.array(labels=='FAKE').astype(np.int)

    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool")
    frame_features = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    for idx, path in enumerate(video_paths):
        # Gather all its frames and add a batch dimension.
        frames = parse_video_into_frames(os.path.join(data_dir, path))
        frames = frames[None, ...]

        # Initialize placeholders to store the masks and features of the current video.
        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                temp_frame_features[i, j, :] = feature_extractor.predict(
                    batch[None, j, :]
                )
            temp_frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

    return (frame_features, frame_masks), labels


### Model Training

In [ ]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(train_sample_metadata,
                   test_size=0.1,
                   random_state=42,
                   stratify=train_sample_metadata['label'])
train_data, train_labels = prepare_data(train_set, "train")
test_data, test_labels = prepare_data(test_set, "test")


In [ ]:
MAX_SEQ_LENGTH = 20
NUM_FEATURES = 2048
DROPOUT = 0.2
frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

x = keras.layers.GRU(32, return_sequences=True)(
    frame_features_input, mask=mask_input)
x = keras.layers.GRU(16)(x)
x = keras.layers.GRU(8)(x)
x = keras.layers.Dropout(DROPOUT)(x)
x = keras.layers.Dense(8, activation="relu")(x)
x = keras.layers.Dense(8, activation="relu")(x)
output = keras.layers.Dense(1, activation="sigmoid")(x)

model = keras.Model([frame_features_input, mask_input], output)

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [ ]:
model.summary()

In [ ]:
EPOCHS = 10
model.fit(
        [train_data[0], train_data[1]],
        train_labels,
        validation_data=([test_data[0], test_data[1]],test_labels),
        epochs=EPOCHS,
        batch_size=8
    )


### Evaluation

In [ ]:
predicted_labels = []
THRESHOLD = 0.5
for idx in range(len(test_data[0])):
  frame_features = test_data[0][idx]
  frame_mask = test_data[1][idx]
  output_prob = model.predict([frame_features, frame_mask])[0]
  if (output_prob >= THRESHOLD):
    predicted_labels.append(1)
  else:
    predicted_labels.append(0)
